In [1]:
# DATASET LINK: http://www.mysignals.gr/research.php
# Relative dataset path dataset\dataset\measurementsGSM_alldata_cells_6065x
import os
import skmob
import pandas as pd
from skmob.measures.individual import *
import folium 
import geopandas as gpd
from skmob.privacy import attacks
from skmob.models.markov_diary_generator import MarkovDiaryGenerator
from skmob.preprocessing import filtering, compression, detection, clustering


dataset = "Input\\Dataset.csv"
aliases_csv = "Input\\aliases.csv"
outputDirectory = "Output"
df = pd.read_csv(dataset, sep=",", header = 0)

# names = ['timestamp','iPhoneUID','dt1','dt2','dt3','dt4','dt5','rssi','rssi1','rssi2','rssi3','rssi4','rssi5','latitude','longitude','finalLatitude','finalLongitude','horizontalAccuracy','isMoving','txPower','cellID','LAC','MNC','ARFCN','freq_dlink','freq_uplink']# dsl_df = distance_straight_line(tdf)
# dsl_rog = radius_of_gyration(tdf)
# dsl_lf = location_frequency(tdf)
# dsl_jl = jump_lengths(tdf)
# dsl_nol = number_of_locations(tdf)
# print(dsl_df.head())
# print(dsl_rog.head())
# print(dsl_lf.head())
# print(dsl_jl.head())
# print(dsl_nol.head())
# print(hl_df.head())


# add all trajectories in a dataframe
tdf = skmob.TrajDataFrame(df, latitude='latitude', longitude='longitude', datetime='timestamp', user_id='iPhoneUID')

print(tdf.head())

##########################################################
# Data preprocessing

# Filter the traj dataframe by removing consecutive locations with speed > 200 kmh
filtered_tdf = filtering.filter(tdf, max_speed_kmh = 200)
num_deleted_points = len(tdf) - len(filtered_tdf)

print("Deleted",num_deleted_points,"out of",len(tdf))

             datetime                               uid   dt1   dt2   dt3  \
0 2012-09-12 17:59:08  a841f74e620f74ec443b7a25d7569545  1.02  1.02  1.01   
1 2012-09-12 17:59:14  a841f74e620f74ec443b7a25d7569545  1.02  1.03  1.02   
2 2012-09-12 17:59:20  a841f74e620f74ec443b7a25d7569545  1.03  1.04  1.02   
3 2012-09-12 17:59:27  a841f74e620f74ec443b7a25d7569545  1.02  1.01  1.04   
4 2012-09-12 17:59:33  a841f74e620f74ec443b7a25d7569545  1.02  1.03  1.02   

    dt4   dt5  rssi  rssi1  rssi2  ...  finalLongitude  horizontalAccuracy  \
0  1.02  1.02   -70    -69    -69  ...       24.027854                65.0   
1  1.02  1.02   -70    -71    -71  ...       24.027854                65.0   
2  1.04  1.02   -70    -71    -71  ...       24.027854                65.0   
3  1.02  1.02   -70    -75    -79  ...       24.027854                65.0   
4  1.02  1.02   -70    -85    -85  ...       24.027946                65.0   

   isMoving  txPower   cellID    LAC  MNC  ARFCN  freq_dlink  freq_u

In [2]:
unique_ids = filtered_tdf.uid.unique().tolist()


In [3]:
print(len(unique_ids))
print(unique_ids)

9
['1e33db5d2be36268b944359fbdbdad21', '22223276ea84bbce3a62073c164391fd', '510635002cb29804d54bff664cab52be', '6882f6cf8c72d6324ba7e6bb42c9c7c2', '7023889b4439d2c02977ba152d6f4c6e', '7cbc37da05801d46e7d80c3b99fd5adb', '8425a81da55ec16b7f9f80c139c235a2', '892d2c3aae6e51f23bf8666c2314b52f', 'a841f74e620f74ec443b7a25d7569545']


In [4]:
group_tdf = filtered_tdf.groupby(['uid']).size()
# print(group_tdf)
out_group_tdf_path = os.path.join(outputDirectory,"uid_count.csv")
temp = pd.DataFrame(group_tdf, columns =['count'])
temp.to_csv(out_group_tdf_path)

filtered_tdf = filtered_tdf[filtered_tdf['uid'] != "510635002cb29804d54bff664cab52be"]

print(filtered_tdf.head())
group_tdf = filtered_tdf.groupby(['uid']).size()
print(group_tdf)

             datetime                               uid   dt1   dt2   dt3  \
0 2013-04-03 18:28:22  1e33db5d2be36268b944359fbdbdad21  1.02  1.05  1.02   
1 2013-04-03 18:28:29  1e33db5d2be36268b944359fbdbdad21  1.06  1.07  1.08   
2 2013-04-03 18:36:55  1e33db5d2be36268b944359fbdbdad21  1.02  1.03  1.02   
3 2013-04-03 18:37:01  1e33db5d2be36268b944359fbdbdad21  1.05  1.12  1.08   
4 2013-04-03 18:37:08  1e33db5d2be36268b944359fbdbdad21  1.02  1.03  1.02   

    dt4   dt5  rssi  rssi1  rssi2  ...  finalLongitude  horizontalAccuracy  \
0  1.02  1.03   -81    -81    -79  ...       24.067551               100.0   
1  1.02  1.02   -81    -79    -75  ...       23.920956               100.0   
2  1.03  1.02   -79    -73    -73  ...       24.019675              1414.0   
3  1.02  1.03   -79    -75    -77  ...       24.019675              1414.0   
4  1.03  1.02   -79    -81    -81  ...       24.019675              1414.0   

   isMoving  txPower   cellID    LAC  MNC  ARFCN  freq_dlink  freq_u

In [5]:
# aliases = pd.read_csv(aliases_csv)
# print(aliases.head())
# aliases = aliases.set_index('uid')
# print(aliases.head())
# # aliases = aliases.T
# # print(aliases.head())
# dict_alias = aliases.to_dict()
# print(dict_alias)

filtered_tdf = filtered_tdf.replace({'1e33db5d2be36268b944359fbdbdad21': 'device_1', '22223276ea84bbce3a62073c164391fd': 'device_2', '6882f6cf8c72d6324ba7e6bb42c9c7c2': 'device_3', '7023889b4439d2c02977ba152d6f4c6e': 'device_4', '7cbc37da05801d46e7d80c3b99fd5adb': 'device_5', '8425a81da55ec16b7f9f80c139c235a2': 'device_6', '892d2c3aae6e51f23bf8666c2314b52f': 'device_7', 'a841f74e620f74ec443b7a25d7569545': 'device_8'})
print(filtered_tdf.head())

             datetime       uid   dt1   dt2   dt3   dt4   dt5  rssi  rssi1  \
0 2013-04-03 18:28:22  device_1  1.02  1.05  1.02  1.02  1.03   -81    -81   
1 2013-04-03 18:28:29  device_1  1.06  1.07  1.08  1.02  1.02   -81    -79   
2 2013-04-03 18:36:55  device_1  1.02  1.03  1.02  1.03  1.02   -79    -73   
3 2013-04-03 18:37:01  device_1  1.05  1.12  1.08  1.02  1.03   -79    -75   
4 2013-04-03 18:37:08  device_1  1.02  1.03  1.02  1.03  1.02   -79    -81   

   rssi2  ...  finalLongitude  horizontalAccuracy  isMoving  txPower   cellID  \
0    -79  ...       24.067551               100.0       0.0   -255.0  60561.0   
1    -75  ...       23.920956               100.0       1.0   -255.0  60561.0   
2    -73  ...       24.019675              1414.0       0.0   -255.0  60565.0   
3    -77  ...       24.019675              1414.0       0.0   -255.0  60565.0   
4    -81  ...       24.019675              1414.0       0.0   -255.0  60565.0   

     LAC  MNC  ARFCN  freq_dlink  freq_uplin

In [6]:

# Calculate radius of gyration
dsl_rog = radius_of_gyration(filtered_tdf)
print(dsl_rog.head(100))
out_dsl_path = os.path.join(outputDirectory,"radius_of_gyration.csv")
dsl_rog.to_csv(out_dsl_path, index = False)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  2.69it/s]

        uid  radius_of_gyration
0  device_1            0.045383
1  device_2            0.258592
2  device_3            0.088066
3  device_4            0.064525
4  device_5            0.668691
5  device_6            0.288058
6  device_7            0.018628
7  device_8            0.478493


In [7]:
# Calculate k-radius of gyration
for k in range(2,21):
    print("|",end="")
    dsl_krog = k_radius_of_gyration(filtered_tdf, k)
    # print(dsl_krog.head(100))
    out_dsl_path = os.path.join(outputDirectory,"k_rog",str(k)+"k_radius_of_gyration.csv")
    dsl_krog.to_csv(out_dsl_path, index = False)
print()

 25%|█████████████████                                                   | 2/8 [00:00<00:00, 18.87it/s]

|

 25%|█████████████████                                                   | 2/8 [00:00<00:00, 18.70it/s]

|

 25%|█████████████████                                                   | 2/8 [00:00<00:00, 17.40it/s]

...

 25%|█████████████████                                                   | 2/8 [00:00<00:00, 19.91it/s]

|

 25%|█████████████████                                                   | 2/8 [00:00<00:00, 18.02it/s]

|

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 12.73it/s]

In [8]:

# Calculate max distance
dsl_md = maximum_distance(filtered_tdf)
print(dsl_md.head(100))
out_dsl_path = os.path.join(outputDirectory,"max_distance.csv")
dsl_md.to_csv(out_dsl_path, index = False)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.60it/s]

        uid  maximum_distance
0  device_1          4.424816
1  device_2          3.389703
2  device_3          1.633196
3  device_4          0.735761
4  device_5          2.453889
5  device_6          1.138429
6  device_7          0.283213
7  device_8          5.719505


In [9]:

# Calculate distinct locations
dsl_dl = number_of_locations(filtered_tdf)
print(dsl_dl.head(100))
out_dsl_path = os.path.join(outputDirectory,"number_of_locations.csv")
dsl_dl.to_csv(out_dsl_path, index = False)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.45it/s]

        uid  number_of_locations
0  device_1                   71
1  device_2                 2114
2  device_3                   22
3  device_4                  124
4  device_5                 1457
5  device_6                  173
6  device_7                   13
7  device_8                 5711


In [10]:

# Calculate location frequency
lf_df = location_frequency(filtered_tdf, normalize = False).reset_index()
print(lf_df.head(10))
out_dsl_path = os.path.join(outputDirectory,"location_frequency.csv")
lf_df.to_csv(out_dsl_path, index = False)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 33.07it/s]

        uid        lat        lng  location_frequency
0  device_1  35.511199  24.023398                9776
1  device_1  35.511223  24.023541                4464
2  device_1  35.511253  24.023679                3765
3  device_1  35.511236  24.023560                3441
4  device_1  35.511266  24.023598                3354
5  device_1  35.511215  24.023512                2698
6  device_1  35.511261  24.023505                1836
7  device_1  35.511302  24.023675                1758
8  device_1  35.511273  24.023577                1673
9  device_1  35.511262  24.023754                1524


In [11]:
# Calculate frequency rank
fr_df = frequency_rank(filtered_tdf)
print(fr_df.head(10))
out_dsl_path = os.path.join(outputDirectory,"frequency_rank.csv")
fr_df.to_csv(out_dsl_path)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 30.90it/s]

                  lat        lng  frequency_rank
uid                                             
device_1 0  35.511199  24.023398               1
         1  35.511223  24.023541               2
         2  35.511253  24.023679               3
         3  35.511236  24.023560               4
         4  35.511266  24.023598               5
         5  35.511215  24.023512               6
         6  35.511261  24.023505               7
         7  35.511302  24.023675               8
         8  35.511273  24.023577               9
         9  35.511262  24.023754              10


In [12]:
# Mean square displacement
import skmob
from skmob.measures.collective import mean_square_displacement
msd = "days,hours,msd\n"
for days in range(1):
    for hours in range(24):
        temp = mean_square_displacement(filtered_tdf, days = 0, hours = 1, minutes = 0, show_progress=False)
        msd += str(days)+","+str(hours)+","+str(temp)+"\n"
    print()
print(msd)
msd_file_path = os.path.join(outputDirectory,"msd.csv")
msd_file = open(msd_file_path, "w")
msd_file.write(msd)
msd_file.close()


days,hours,msd
0,0,3.0862160590423033
0,1,3.0862160590423033
0,2,3.0862160590423033
0,3,3.0862160590423033
0,4,3.0862160590423033
0,5,3.0862160590423033
0,6,3.0862160590423033
0,7,3.0862160590423033
0,8,3.0862160590423033
0,9,3.0862160590423033
0,10,3.0862160590423033
0,11,3.0862160590423033
0,12,3.0862160590423033
0,13,3.0862160590423033
0,14,3.0862160590423033
0,15,3.0862160590423033
0,16,3.0862160590423033
0,17,3.0862160590423033
0,18,3.0862160590423033
0,19,3.0862160590423033
0,20,3.0862160590423033
0,21,3.0862160590423033
0,22,3.0862160590423033
0,23,3.0862160590423033



In [13]:
# Random entropy
import skmob
from skmob.measures.individual import random_entropy

re_df = random_entropy(filtered_tdf)
print(re_df.head())
re_df_path = os.path.join(outputDirectory,"random_entropy.csv")
re_df.to_csv(re_df_path, index = False)

100%|████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.83it/s]

        uid  random_entropy
0  device_1        6.149747
1  device_2       11.045760
2  device_3        4.459432
3  device_4        6.954196
4  device_5       10.508785


In [14]:
# Home and work attack

import skmob
from skmob.privacy import attacks
from skmob.core.trajectorydataframe import TrajDataFrame

at = attacks.HomeWorkAttack()
r = at.assess_risk(filtered_tdf)
print(r)

hw_attack_path = os.path.join(outputDirectory,"home_work_attack.csv")
r.to_csv(hw_attack_path, index = False)

        uid  risk
0  device_1   1.0
1  device_2   1.0
2  device_3   1.0
3  device_4   1.0
4  device_5   1.0
5  device_6   1.0
6  device_7   1.0
7  device_8   1.0


In [15]:
moving_tdf = filtered_tdf[filtered_tdf['isMoving'] == 1]
# print(moving_tdf.head())
moving_tdf = moving_tdf.groupby(['uid']).size()
print(moving_tdf.head())
# print(group_tdf)
out_group_tdf_path = os.path.join(outputDirectory,"moving_uid_count.csv")
temp = pd.DataFrame(moving_tdf, columns =['count'])
temp.to_csv(out_group_tdf_path)

uid
device_1     458
device_2    2034
device_3     914
device_4      82
device_5    1282
dtype: int64


In [19]:
import skmob
import folium

    return distance

csv_Freq_rank = "Output\\frequency_rank.csv"

# filtered_tdf
hour_tdf = filtered_tdf
hour_tdf = hour_tdf.set_index('datetime')
hour_tdf = hour_tdf[hour_tdf.index.dayofweek < 5]
hour_tdf= hour_tdf.between_time("11:00:00","15:00:00")
hour_tdf = hour_tdf.reset_index()
hour_tdf = hour_tdf.rename(columns={"index" : "datetime"})
hour_tdf.to_csv("Output\\trajectories_hour_limited.csv", index = False)
print(hour_tdf.head())


             datetime       uid   dt1   dt2   dt3   dt4   dt5  rssi  rssi1  \
0 2013-04-08 11:00:04  device_1  1.01  1.03  1.02  1.02  1.02   -79    -75   
1 2013-04-08 11:00:10  device_1  1.01  1.02  1.02  1.02  1.02   -79    -73   
2 2013-04-08 11:00:16  device_1  1.02  1.02  1.02  1.02  1.02   -79    -73   
3 2013-04-08 11:00:22  device_1  1.02  1.02  1.02  1.02  1.02   -79    -75   
4 2013-04-08 11:00:29  device_1  1.02  1.02  1.02  1.02  1.02   -79    -73   

   rssi2  ...  finalLongitude  horizontalAccuracy  isMoving  txPower   cellID  \
0    -75  ...       24.023598                65.0       0.0   -255.0  60562.0   
1    -73  ...       24.023598                65.0       0.0   -255.0  60562.0   
2    -75  ...       24.023598                65.0       0.0   -255.0  60562.0   
3    -75  ...       24.023598                65.0       0.0   -255.0  60562.0   
4    -73  ...       24.023598                65.0       0.0   -255.0  60562.0   

     LAC  MNC  ARFCN  freq_dlink  freq_uplin

In [23]:
# Calculate  frequency rank

in_hour_tdf = skmob.TrajDataFrame(hour_tdf, latitude = 'lat', longitude = 'long', user_id = 'uid', datetime = 'datetime')

print(in_hour_tdf.head())

fr_df = frequency_rank(in_hour_tdf)
# print(fr_df.head(10))
out_dsl_path = os.path.join(outputDirectory,"frequency_rank_work_hours.csv")
fr_df.to_csv(out_dsl_path)

fr_df['frequency_rank']=fr_df['frequency_rank'].astype(int)
# print(df.head())
fr_df = fr_df.loc[fr_df['frequency_rank'] < 100]
fr_df.to_csv("Output\\frequency_rank_work_hours_top3.csv")
print(fr_df.head())

# frequency_rank = folium.Map(location=[35.5,24.0], zoom_start = 11)
# for index, row in fr_df.iterrows():
#     folium.Marker( location=[ row[1], row[2] ], fill_color='#43d9de', radius=8, popup=row[0]).add_to( frequency_rank )
# # frequency_rank

# # filtered_tdf

100%|███████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 124.98it/s]

             datetime       uid   dt1   dt2   dt3   dt4   dt5  rssi  rssi1  \
0 2013-04-08 11:00:04  device_1  1.01  1.03  1.02  1.02  1.02   -79    -75   
1 2013-04-08 11:00:10  device_1  1.01  1.02  1.02  1.02  1.02   -79    -73   
2 2013-04-08 11:00:16  device_1  1.02  1.02  1.02  1.02  1.02   -79    -73   
3 2013-04-08 11:00:22  device_1  1.02  1.02  1.02  1.02  1.02   -79    -75   
4 2013-04-08 11:00:29  device_1  1.02  1.02  1.02  1.02  1.02   -79    -73   

   rssi2  ...  finalLongitude  horizontalAccuracy  isMoving  txPower   cellID  \
0    -75  ...       24.023598                65.0       0.0   -255.0  60562.0   
1    -73  ...       24.023598                65.0       0.0   -255.0  60562.0   
2    -75  ...       24.023598                65.0       0.0   -255.0  60562.0   
3    -75  ...       24.023598                65.0       0.0   -255.0  60562.0   
4    -73  ...       24.023598                65.0       0.0   -255.0  60562.0   

     LAC  MNC  ARFCN  freq_dlink  freq_uplin